# Github Acquired, Exploring the Past Acquisitions !!  
<br>

Microsoft recently acquired Github for a whopping 7.5 Billion dollars which is one of the interesting acquisitions made in the history of Microsoft. It will be interesting to see how this acquisition goes in future years. Given all this buzz on social media, I thought it’d be interesting to explore and visualize the data on acquisitions by the top seven companies since past four decadaes. I collated the dataset of acquisitions by Amazon, Apple, Facebook, Google/Alphabet, IBM, Yahoo and Microsoft.  The attributes in the dataset includes the date, year, month of the acquisition, name of the company acquired, value or the cost of acquisition, business use-case of the acquisition, and the country from which the acquisition was made. In this notebook, I have explored this dataset and visualized all the past acquisitions by these companies.  

Dataset Link : https://www.kaggle.com/shivamb/company-acquisitions-7-top-companies 







### <a>Contents</a> <br>  <br>  


<a href="#1">1. Microsoft Acquisitions</a>      

<a href="#2">2. Google Acquisitions  </a>  

<a href="#3">3. IBM Acquisitions  </a>  

<a href="#4">4. Yahoo Acquisitions  </a>  

<a href="#5">5. Apple Acquisitions  </a>  

<a href="#6">6. Twitter Acquisitions  </a>  

<a href="#7">7. Facebook Acquisitions  </a>  

<a href="#8">8. Business Use-Cases of Acquired Companies  </a>  

**Please Note -** In this dataset many acquisition amounts are not disclosed publicly by the parent companies. So all the graphs related to cost includes only the companies having publicly disclosed amounts.

In [2]:
## load the libraries 
from plotly.offline import init_notebook_mode, iplot
from wordcloud import WordCloud, STOPWORDS
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly import tools
from datetime import date
import pandas as pd
import numpy as np 
import seaborn as sns
import random 
import warnings
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)
df = pd.read_csv("../input/acquisitions.csv")
import squarify
from collections import Counter


## bar plots horizontal
def bar_hor(df, col, title, color, w=None, h=None, lm=0, limit=100, return_trace=False, rev=False):
    cnt_srs = df[col].value_counts()
    yy = cnt_srs.head(limit).index[::-1] 
    xx = cnt_srs.head(limit).values[::-1] 
    if rev:
        yy = cnt_srs.tail(limit).index[::-1] 
        xx = cnt_srs.tail(limit).values[::-1] 
        
    
    trace = go.Bar(y=yy, x=xx, orientation = 'h', marker=dict(color=color, opacity=0.8))
    if return_trace:
        return trace 
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

## bar plots horizontal with x,y
def bar_hor_noagg(x, y, title, color, w=None, h=None, lm=0, limit=100, rt=False):
    trace = go.Bar(y=x, x=y, orientation = 'h', marker=dict(color=color, opacity=0.8))
    if rt:
        return trace
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

## bar plots verticals with x and y 
def bar_ver_noagg(x, y, title, color, w=None, h=None, lm=0, rt = False):
    trace = go.Bar(y=y, x=x, marker=dict(color=color, opacity=0.8))
    if rt:
        return trace
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
## Generating all visuals for a company 
def generate_all(compdf, compname, color):
    t = compdf[['Company', 'AcquisitionYear']].groupby('AcquisitionYear').agg({'Company' : lambda x : "|".join(x)})
    x = []
    y = []
    z = []
    s = []
    for i, each in enumerate(t.index):
        x1 = each
        y1 = t.values[i][0].split("|")
        for j,comp in enumerate(y1):
            x.append(x1)
            y.append(j+3)
            p = compdf[compdf['Company'] == comp]['Value (USD)'].iloc(0)[0]
            if str(p).lower() == "nan":
                z.append("Company: " + comp)
            else:
                z.append("Company: " + comp +" <br> Amount: $"+ str(int(p)))
            if p > 1000000000:
                s.append(23)
            elif p > 50000000:
                s.append(21)
            elif p > 25000000:
                s.append(19)
            elif p > 12500000:
                s.append(17)
            elif p > 6500000:
                s.append(15)
            elif p > 25000:
                s.append(13)
            else:
                s.append(10)

    trace1 = go.Scatter(x=x, y=y, mode='markers', text=z, marker=dict(size=s, color=color))
    data = [trace1]
    layout = go.Layout(title="All acquisitions By " + compname,  yaxis=dict(
            autorange=True,
            showgrid=False,
            zeroline=False,
            showline=False,
            autotick=True,
            ticks='',
            showticklabels=False
        ), height=600)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    

## Generate the Bars for all companies 
def bars():
    ## all
    costly = compdf.sort_values('Value (USD)', ascending = False)
    x = costly['Company'][:15]
    y = costly['Value (USD)'][:15]
    tr1 = bar_ver_noagg(x, y, "Top "+compname+" Acquisitions by Cost",compcol, h=400, lm=50, rt = False)

    t = dict(compdf['AcquisitionYear'].value_counts())
    x = list(t.keys())
    y = list(t.values())
    tr1 = bar_ver_noagg(x[::-1], y[::-1], "Number of Acquisitions by "+compname+" per Year",compcol, h=500, lm=40, rt=True)



    t = dict(compdf['AcquisitionMonth'].value_counts())
    ordd = ['January', 'February', 'March','April','May','June','July','August','September', 'October', 'November','December']
    y1 = [t[i] for i in ordd] 
    tr2 = bar_ver_noagg(ordd, y1, "Number of Acquisitions by "+compname+" Every Month",compcol, h=400, lm=40, rt=True)

    fig = tools.make_subplots(rows=1, cols=2, print_grid=False, subplot_titles = [compname+' Acquisitions - Per Month',compname+' Acquisitions - Per Year' ])
    fig.append_trace(tr2, 1, 1);
    fig.append_trace(tr1, 1, 2);
    fig['layout'].update(height=400, showlegend=False);
    iplot(fig); 


    col = "AcquisitionMonth"
    data = []
    for each in compdf[col].value_counts().index:
        y1 = compdf[compdf[col]==each]['Value (USD)'].dropna()
        trace0 = go.Box(y=y1, name = each, marker = dict(color = compcol))
        data.append(trace0)

    layout = dict(title="Acquisition Cost per Month - "+compname, height=400, showlegend=False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

# treemap 1
def createhmap1(compcol):
    t = compdf['Country'].value_counts()
    keys = t.index[1:] 
    vals = t.values[1:]

    x = 0.
    y = 0.
    width = 100.
    height = 100.
    colcnt = 0
    values = vals

    normed = squarify.normalize_sizes(values, width, height)
    rects = squarify.squarify(normed, x, y, width, height)

    color_brewer = [compcol]
    shapes = []
    annotations = []
    counter = 0

    for r in rects:
        shapes.append( 
            dict(
                type = 'rect', 
                x0 = r['x'], 
                y0 = r['y'], 
                x1 = r['x']+r['dx'], 
                y1 = r['y']+r['dy'],
                line = dict( width = 5, color="#fff" ),
                fillcolor = color_brewer[colcnt]
            ) 
        )
        annotations.append(
            dict(
                x = r['x']+(r['dx']/2),
                y = r['y']+(r['dy']/2),
                text = list(keys)[counter] +" ("+ str(values[counter]) + ")",
                showarrow = False
            )
        )
        counter = counter + 1
        colcnt+=1
        if colcnt >= len(color_brewer):
            colcnt = 0

    # For hover text
    trace0 = go.Scatter(
        x = [ r['x']+(r['dx']/2) for r in rects ], 
        y = [ r['y']+(r['dy']/2) for r in rects ],
        text = [ str(v)+" ("+str(values[k])+" )" for k,v in enumerate(keys) ], 
        mode = 'text',
    )

    layout = dict(
        height=450, 
        xaxis=dict(
                autorange=True,
                showgrid=False,
                zeroline=False,
                showline=False,
                autotick=True,
                ticks='',
                showticklabels=False
            ),
        yaxis=dict(
                autorange=True,
                showgrid=False,
                zeroline=False,
                showline=False,
                autotick=True,
                ticks='',
                showticklabels=False
            ),
        shapes=shapes,
        annotations=annotations,
        hovermode='closest',
        title="Countries (Apart from USA) from where " + compname + " acquired"
    )

    figure = dict(data=[trace0], layout=layout)
    iplot(figure, filename='squarify-treemap')

## treemap 2 
def createhmap2(compcol):
    txt = " ".join(compdf['Business']).lower()
    wrds = txt.split()
    words = []
    for each in wrds:
        if each.endswith("s"):
            words.append(each[:-1])
        else:
            words.append(each)
    t = Counter(words)
    m = [x for x in t.most_common(30) if x[0] not in STOPWORDS]

    keys = [m1[0] for m1 in m]
    vals = [m1[1] for m1 in m] 

    x = 0.
    y = 0.
    width = 100.
    height = 100.
    colcnt = 0
    values = vals

    normed = squarify.normalize_sizes(values, width, height)
    rects = squarify.squarify(normed, x, y, width, height)

    color_brewer = [compcol]
    shapes = []
    annotations = []
    counter = 0

    for r in rects:
        shapes.append( 
            dict(
                type = 'rect', 
                x0 = r['x'], 
                y0 = r['y'], 
                x1 = r['x']+r['dx'], 
                y1 = r['y']+r['dy'],
                line = dict( width = 5, color="#fff" ),
                fillcolor = color_brewer[colcnt]
            ) 
        )
        annotations.append(
            dict(
                x = r['x']+(r['dx']/2),
                y = r['y']+(r['dy']/2),
                text = list(keys)[counter] +" ("+ str(values[counter]) + ")",
                showarrow = False
            )
        )
        counter = counter + 1
        colcnt+=1
        if colcnt >= len(color_brewer):
            colcnt = 0

    # For hover text
    trace0 = go.Scatter(
        x = [ r['x']+(r['dx']/2) for r in rects ], 
        y = [ r['y']+(r['dy']/2) for r in rects ],
        text = [ str(v)+" ("+str(values[k])+" )" for k,v in enumerate(keys) ], 
        mode = 'text',
    )

    layout = dict(
        height=450, 
        xaxis=dict(
                autorange=True,
                showgrid=False,
                zeroline=False,
                showline=False,
                autotick=True,
                ticks='',
                showticklabels=False
            ),
        yaxis=dict(
                autorange=True,
                showgrid=False,
                zeroline=False,
                showline=False,
                autotick=True,
                ticks='',
                showticklabels=False
            ),
        shapes=shapes,
        annotations=annotations,
        hovermode='closest',
        title="Business Use-Cases of Acquired Companies by " + compname
    )

    figure = dict(data=[trace0], layout=layout)
    iplot(figure, filename='squarify-treemap')

<br><hr>
# <a id="1"><font color="#bd4ff9">1. Acquisitions by Microsoft</font></a>  <br><br>

Microsoft is an American public multinational corporation headquartered in Redmond, Washington, USA that develops, manufactures, licenses, and supports a wide range of products and services predominantly related to computing through its various product divisions. Established on April 4, 1975, Microsoft rose to dominate the home computer operating system market with MS-DOS in the mid-1980s, followed by the Microsoft Windows line of operating systems, Microsoft Office, video games, consumer electronics and digital services markets 

In [3]:
compcol = "#bd4ff9"
compname = "Microsoft"
compdf = df[df['ParentCompany'] == compname] 
generate_all(compdf, compname, compcol)
bars()
createhmap1(compcol)

> - Microsoft made its first acquisition on **June 29, 1987** and acquired the company named **Forethought**. As of 2018 June and its latest acquisition is GitHub which  was acquired on June 4th for 7.5B US dollars.    
> - Most costly acquisitions by Microsoft are : **Linkedin (26.2 B), Skype (8.5 B), and Github (7.5 B**). Maximum number of acquisitions were made in the years 2015 and 2006 with 19 and 18 acquisitions respectively.   
> - Microsoft has made total 210 acquisitions, **68% of them were from USA** only, while remaining 32% acquisitions were mainly from Canada (17 acqusitions), Israil (13 acquisitions), and UK (7 acquisitions)    
> - Microsoft has purchased an average of **six companies a year**. and they purchased more than ten companies a year between 2005 and 2008.  
> - Microsoft has made **ten acquisitions worth over one billion dollars**: Skype (2011), aQuantive (2007), Fast Search & Transfer (2008), Navision (2002), Visio Corporation (2000), Yammer (2012), Nokia (2013), Mojang (2014), LinkedIn (2016) and GitHub (2018).


<br><hr>
# <a id="2"><font color="#fc2054">2. Acquisitions by Google</font></a>  <br><br>



In [4]:
compcol = "#fc2054"
compname = "Google"
compdf = df[df['ParentCompany'] == compname] 
generate_all(compdf, compname, compcol)
bars()
createhmap1(compcol)

> - Google made its first acquisition on **Feb 12, 2001** and acquired the company named **Deja**. As of 2018 June, its latest acquisition is Tenor which is a GIF image search tool.        
> - Google's focus on Android and Mobile was always higher. Till date, **Motorola Mobility** remains the most expensive purchase amounting to $12.5 Billion, followed by **NestLabs (3.2 B), DoubleClick (3.1 B) and Youtube (1.65 B)** among other dear acquisitions.   
> - Google procured the companies under disparate categories such as **Mobile, AI, Cloud, Robotics, Maps, Communication, Search and Publication**.   
> - Eric Schmidt served as the the **CEO for Google for almost a decade** (2001-2011) before annoucing the end of his tenure on January 20, 2011. Larry Page was appointed as the new CEO and served the position till 2015 before passing the torch to **Sundar Pichai in late 2015**. Page moved to takecare of Google's new entitiy and its parent company - Alphabet Inc.    
> - Though Schmidt served the longest, number of acquisitions under him were least (33) as compared to acquistions by his successors Larry and Sundar. Larry Page initially acquiried companies in Social space to protect and grow Google+. But around 2013, he began started laying the ground work for Alphabet, acquired companies in **AI, Home Automation and IOT**. With about 90 acquisitions in just under 2 years, Sundar Pichai is the fastest CEO in terms of acquiring the companies.   
> - Google has acquired the largest number of AI companies among other tech giants such as Microsoft, Apple and Facebook. Major **AI acquisions by Google** include DeepMind, **Kaggle Inc** and API.ai. Majority of them were acquired in the period of 2013-2016  
> - Google has made **total 215 (as of June 2018) acquisition**s, 74 percent of them were from USA only, while remaining **26 percent** acquisitions were majorly from **Canada (11 acqusitions), UK (7 acquisitions), and Germany (5 acquisitions)**      


<br><hr>
# <a id="3"><font color="#66ff87">3. Acquisitions by IBM</font></a>  <br><br>


In [5]:
compcol = "#66ff87"
compname = "IBM"
compdf = df[df['ParentCompany'] == compname] 
generate_all(compdf, compname, compcol)
bars()
createhmap1(compcol)

> - IBM has acquired about **162 companies as of June 2018**, with 7 companies acquired on an average every year since 2010.  They made their first acquisition in 2001 by acquisiring the company named **Mainspring**, their latest acquisition is Vivant Digital which got acquired in 2017.  
> - IBM's most costly acquisitions are **Congos (5 B USD), PWC Consulting (3.5 B), Truven Health Analytics (2.6 B USD)**, and Rational Software Corporation (2.1 B USD)  
> - The number of acqusitions every year is decreasing as observed from the graph, maximum number of acquistions in any year by IBM is 15 for years - **2006 and 2010**.  No acquisition has been observed as of 2018  
> - Apart from US, **Canada, Israil, India, Germany, and UK** are the countries from where IBM has purchased the companies  

<br><hr>
# <a id="4"><font color="#7b86ed">4. Acquisitions by Yahoo</font></a>  <br><br>

 Yahoo! is a computer software and web search engine company founded on March 1, 1995. The company is a public corporation and its headquarters is located in Sunnyvale, California. It was founded by Stanford University graduate students Jerry Yang and David Filo in 1994. According to web traffic analysis companies, Yahoo has been one of the most visited websites on the Internet, with more than 130 million unique users per month in the United States alone.

In [8]:
compcol = "#7b86ed"
compname = "Yahoo"
compdf = df[df['ParentCompany'] == compname] 
generate_all(compdf, compname, compcol)
bars()
createhmap1(compcol)

> - Yahoo's first acquisition was the purchase of **Net Controls**, a web search engine company, in September 1997 for **USD 1.4 Million**. As of April 2008, the company's largest acquisition is the purchase of **Broadcast.com**, an Internet radio company, for **USD 5.7 billion**, making Broadcast.com co-founder Mark Cuban a billionaire. Most of the companies acquired by Yahoo were based in the United States; **78 of the companies are from the United States**, and 15 are based in a foreign country. As of July 2015, Yahoo has acquired **114 companies, with Polyvore being the latest**.  
> - Number of acquisitions by Yahoo were very less till 2012 close to **2 or 3 acquisitions per year**, but it is interesting to note that **in 2013, they acquired 27 comapnies** which is maximum number of companies acquired by any company in single year  

<br><hr>
# <a id="5"><font color="#f26a93">5. Acquisitions by Apple</font></a>  <br><br>


In [9]:
compcol = "#f26a93"
compname = "Apple"
compdf = df[df['ParentCompany'] == compname] 
generate_all(compdf, compname, compcol)
bars()
createhmap1(compcol)

> - As of June 2018, Apple is publicly known to have acquired **72 companie**s. The actual number of acquisitions is possibly larger as Apple does not reveal the majority of its acquisitions unless discovered by journals. According to Wikipedia, **Apple has not released the financial details for the majority of its mergers and acquisitions**.
> - The company made its first acquisition on **March 2, 1988**, with its purchase of **Network Innovations**. In 2013, Apple acquired fifteen companies—more than it has in any other year so far. Apple's largest acquisition was that of **Beats Electronics** in August 2014 for **USD 3 billion**. Of the companies Apple has acquired, **49 were based in the United States**.

<br><hr>
# <a id="6"><font color="orange">6. Acquisitions by Twitter</font></a>  <br><br>


In [6]:
compcol = "orange"
compname = "Twitter"
compdf = df[df['ParentCompany'] == compname] 
generate_all(compdf, compname, compcol)
bars()
createhmap1(compcol)

> - Twitter acquired **summarize** in 2008 and their last acquisition was observed in 2016, ie. no acquisition is observed since past 2 years.  
> - Twitter's most costly acquisitions wre **TellApart (479 Million), MoPub (350 Million), and Magic Pony Technology (150 Million)**    
> - In the year 2014, they made highest number of acquisitions equal to 11, on an average the number of acquisitions per year by twitter is close to 6 acquisitions per year.  
> - Overall they have acquired only **53 companies, and 90 percent of them were from USA**    

<br><hr>
# <a id="7"><font color="#40fcdc">7. Acquisitions by Facebook</font></a>  <br><br>


In [7]:
compcol = "#40fcdc"
compname = "Facebook"
compdf = df[df['ParentCompany'] == compname] 
generate_all(compdf, compname, compcol)
bars()
createhmap1(compcol)

> - Facebook acquired **facebook.com domain name** in August 23, 2005 as thier first purchase, as of 2018 june they have acquired a total of **67 companies**, out of which **51 are from USA**. 
    > - Their most costly acquisition was **WhatsApp** acquired on February 19, 2014 for a massive **19 Billion dollars**. **Oculus VR** and **Instagram** were other popular acquisitions with cost equal to **2B and 1B dollars** respectively.  They made highest number of acquisitions in **2011 equal to 11.** 
> - The majority of the companies acquired by Facebook are based in the **United States**, and in turn, a large percentage of these companies are based in or around the **San Francisco Bay Area**.  
> - Most of Facebook's acquisitions have primarily been **'talent acquisitions'** and acquired products are often shut-down. 


> > "We have not once bought a company for the company. We buy companies to get excellent people... In order to have a really entrepreneurial culture one of the key things is to make sure we're recruiting the best people. One of the ways to do this is to focus on acquiring great companies with great founders." - **Mark Zuckerberg**

<br><hr>
# <a id="8"><font color="red">8. Comparing the Business UseCases of Acquired Companies</font></a>  <br><br>

In this section, lets compare the business use cases of the acquired companies.  

In [ ]:
## Business Keywords - Microsoft
compcol = "#bd4ff9"
compname = "Microsoft"
compdf = df[df['ParentCompany'] == compname] 
createhmap2(compcol)

## Business Keywords - Google
compcol = "#fc2054"
compname = "Google"
compdf = df[df['ParentCompany'] == compname] 
createhmap2(compcol)

## Business Keywords - IBM
compcol = "#66ff87"
compname = "IBM"
compdf = df[df['ParentCompany'] == compname] 
createhmap2(compcol)

## Business Keywords - Yahoo
compcol = "#7b86ed"
compname = "Yahoo"
compdf = df[df['ParentCompany'] == compname] 
createhmap2(compcol)

## Business Keywords - Twitter
compcol = "orange"
compname = "Twitter"
compdf = df[df['ParentCompany'] == compname] 
createhmap2(compcol)

## Business Keywords - Apple
compcol = "#f26a93"
compname = "Apple"
compdf = df[df['ParentCompany'] == compname] 
createhmap2(compcol)

## Business Keywords - Facebook
compcol = "#40fcdc"
compname = "Facebook"
compdf = df[df['ParentCompany'] == compname] 
createhmap2(compcol)

Top usecases of acquired companies for respective parent companies are:  

> - **Microsoft :** Software, Internet, Applications, Management, Mobile 
> - **Google :** Mobile, Software, Social, Video, Search 
> - **IBM :** Software, Management, Analytics, Data 
> - **Yahoo :** Social, Online, Video, Advertising 
> - **Twitter :** Social, Mobile 
> - **Apple :** Software, Music, Recognization (Face / Voice)
> - **Facebook :** App, Social, Mobile

## Thanks for viewing, share your feedback, suggestions in the comments. 
### Please consider upvoting the notebook if you liked it.